<a href="https://colab.research.google.com/github/Midhilesh4890/House-Price-Prediction-Using-Pytorch/blob/main/Pytorch_Advanced_Housing_Prices_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df=pd.read_csv('/content/houseprice.csv',usecols=["SalePrice", "MSSubClass", "MSZoning", "LotFrontage", "LotArea",
                                         "Street", "YearBuilt", "LotShape", "1stFlrSF", "2ndFlrSF"]).dropna()

In [ ]:
df.head()

In [ ]:
for i in df.columns:
    print("Column name {} and unique values are {}".format(i,len(df[i].unique())))

In [ ]:
from datetime import datetime
datetime.now().year

In [ ]:
df['Total Years'] = datetime.now().year - df['YearBuilt']

In [ ]:
df.drop('YearBuilt',axis=1,inplace = True)

In [ ]:
df.columns

In [ ]:
cat_features=["MSSubClass", "MSZoning", "Street", "LotShape"]
out_feature=["SalePrice"]

In [ ]:
# for i in cat_features:
#   print(i,df[i].nunique())

cat_dims = [(df[col].nunique()) for col in cat_features]
cat_dims

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
lbl_encoders = {}
for feature in cat_features:
  lbl_encoders[feature] = LabelEncoder()
  df[feature] = lbl_encoders[feature].fit_transform(df[feature])

In [ ]:
df.head()

In [ ]:
cat_features = np.stack([df['MSSubClass'],df['MSZoning'],df['Street'],df['LotShape']],1)

In [ ]:
cat_features

In [ ]:
import torch
cat_features = torch.tensor(cat_features,dtype=torch.int64)
cat_features

In [ ]:
cont_features=[]
for i in df.columns:
    if i in ["MSSubClass", "MSZoning", "Street", "LotShape","SalePrice"]:
        pass
    else:
        cont_features.append(i)

In [ ]:
cont_features

In [ ]:
cont_values=np.stack([df[i].values for i in cont_features],axis=1)
cont_values=torch.tensor(cont_values,dtype=torch.float)
cont_values

In [ ]:
y = torch.tensor(df['SalePrice'].values,dtype=torch.float).reshape(-1,1)
y

In [ ]:
 cat_dims = [len(df[col].unique()) for col in ["MSSubClass", "MSZoning", "Street", "LotShape"]]
 cat_dims

In [ ]:
embedding_dms = [(x,min(50,(x+1)//2)) for x in cat_dims]

In [ ]:
embedding_dms

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
embed_representation = nn.ModuleList([nn.Embedding(inp,op) for inp,op in embedding_dms])
embed_representation

In [ ]:
cat_featuresz = cat_features[0:4]
cat_featuresz

In [ ]:
pd.set_option('display.max_rows',500)
embedding_val = []
for i,e in enumerate(embed_representation):
  embedding_val.append(e(cat_features[:,i]))

In [ ]:
embedding_val

In [ ]:
z = torch.cat(embedding_val,axis=1)

In [ ]:
z

In [ ]:
dropout = nn.Dropout(0.4)

In [ ]:
final_embed = dropout(z)
final_embed

In [ ]:
##### Create a Feed Forward Neural Network
import torch
import torch.nn as nn
import torch.nn.functional as F
class FeedForwardNN(nn.Module):

    def __init__(self, embedding_dim, n_cont, out_sz, layers, p=0.5):
        super().__init__()
        self.embeds = nn.ModuleList([nn.Embedding(inp,out) for inp,out in embedding_dim])
        self.emb_drop = nn.Dropout(p)
        self.bn_cont = nn.BatchNorm1d(n_cont)
        
        layerlist = []
        n_emb = sum((out for inp,out in embedding_dim))
        n_in = n_emb + n_cont
        
        for i in layers:
            layerlist.append(nn.Linear(n_in,i)) 
            layerlist.append(nn.ReLU(inplace=True))
            layerlist.append(nn.BatchNorm1d(i))
            layerlist.append(nn.Dropout(p))
            n_in = i
        layerlist.append(nn.Linear(layers[-1],out_sz))
            
        self.layers = nn.Sequential(*layerlist)
    
    def forward(self, x_cat, x_cont):
        embeddings = []
        for i,e in enumerate(self.embeds):
            embeddings.append(e(x_cat[:,i]))
        x = torch.cat(embeddings, 1)
        x = self.emb_drop(x)
        
        x_cont = self.bn_cont(x_cont)
        x = torch.cat([x, x_cont], 1)
        x = self.layers(x)
        return x

In [ ]:
torch.manual_seed(100)
model=FeedForwardNN(embedding_dms,len(cont_features),1,[100,50],p=0.4)

In [ ]:
model

In [ ]:
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

In [ ]:
batch_size=1200
test_size=int(batch_size*0.15)
train_categorical=cat_features[:batch_size-test_size]
test_categorical=cat_features[batch_size-test_size:batch_size]
train_cont=cont_values[:batch_size-test_size]
test_cont=cont_values[batch_size-test_size:batch_size]
y_train=y[:batch_size-test_size]
y_test=y[batch_size-test_size:batch_size]

In [ ]:
len(train_categorical),len(test_categorical),len(train_cont),len(test_cont),len(y_train),len(y_test)

In [ ]:

epochs=5000
final_losses=[]
for i in range(epochs):
    i=i+1
    y_pred=model(train_categorical,train_cont)
    loss=torch.sqrt(loss_function(y_pred,y_train)) ### RMSE
    final_losses.append(loss)
    if i%10==1:
        print("Epoch number: {} and the loss : {}".format(i,loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(range(epochs), final_losses)
plt.ylabel('RMSE Loss')
plt.xlabel('epoch')

In [ ]:
y_pred=""
with torch.no_grad():
    y_pred=model(test_categorical,test_cont)
    loss=torch.sqrt(loss_function(y_pred,y_test))
print('RMSE: {}'.format(loss))

In [ ]:
data_verify=pd.DataFrame(y_test.tolist(),columns=["Test"])

In [ ]:
data_predicted=pd.DataFrame(y_pred.tolist(),columns=["Prediction"])

In [ ]:
data_predicted

In [ ]:
final_output=pd.concat([data_verify,data_predicted],axis=1)
final_output['Difference']=final_output['Test']-final_output['Prediction']
final_output.head()

In [ ]:
#### Saving The Model
#### Save the model
torch.save(model,'HousePrice.pt')

In [ ]:
torch.save(model.state_dict(),'HouseWeights.pt')

In [ ]:
### Loading the saved Model
embs_size=[(15, 8), (5, 3), (2, 1), (4, 2)]
model1=FeedForwardNN(embs_size,5,1,[100,50],p=0.4)

In [ ]:
model1.load_state_dict(torch.load('HouseWeights.pt'))

In [ ]:
model1.eval()